In [9]:
##############################
### LIBRERÍAS NECESARIAS ####
##############################

import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact
from sklearn import neighbors
import joblib
from google.colab import drive
import sys
import os

In [21]:
#Conectar al drive local
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

In [24]:
##############################
### 0. CONEXIÓN A BASE DE DATOS ###
##############################

con = sql.connect('data/db_movies_c1') #Conectarse a la base de datos existente y transportar datos
cur = con.cursor() #Otra conexión (cursor) para ejecutar las consultas en la bd sin traer ni llevar info

# Verifica tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())  # Deberías ver 'full_ratings' en la salida


[('ratings',), ('movies',), ('movies_clean',), ('full_ratings',)]


In [43]:
#############################################
### 1. CONSULTAR LA TABLA full_ratings ######
#############################################

# Cargar la tabla 'ratings' en un DataFrame
ratings = pd.read_sql("SELECT * FROM ratings", con)

# Cargar la tabla 'full_ratings' en un DataFrame
df_peliculas = pd.read_sql("SELECT * FROM full_ratings", con)

# Información de la tabla 'full_ratings' (df_peliculas)
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movieId      1104 non-null   int64  
 1   title        1104 non-null   object 
 2   year         1104 non-null   int64  
 3   Action       1104 non-null   int64  
 4   Adventure    1104 non-null   int64  
 5   Animation    1104 non-null   int64  
 6   Children     1104 non-null   int64  
 7   Comedy       1104 non-null   int64  
 8   Crime        1104 non-null   int64  
 9   Film-Noir    1104 non-null   int64  
 10  Horror       1104 non-null   int64  
 11  IMAX         1104 non-null   int64  
 12  Musical      1104 non-null   int64  
 13  Mystery      1104 non-null   int64  
 14  Romance      1104 non-null   int64  
 15  Sci-Fi       1104 non-null   int64  
 16  Thriller     1104 non-null   int64  
 17  War          1104 non-null   int64  
 18  Western      1104 non-null   int64  
 19  avg_ra

In [27]:
#######################################################
### 2. PREPARACIÓN DE DATOS PARA MODELOS DE CONTENIDO
#######################################################

# Escalar el año
sc = MinMaxScaler()
df_peliculas["year_scaled"] = sc.fit_transform(df_peliculas[["year"]])

# Seleccionar columnas relevantes para el modelo
columnas_usadas = list(df_peliculas.columns[3:-2]) + ['year_scaled']  # géneros + year
df_dum2 = df_peliculas[columnas_usadas]

# Guardar para uso posterior
#joblib.dump(df_dum2, "salidas\\movies_dum2.joblib")

In [28]:
#####################################################################
### 3. RECOMENDACIÓN MANUAL BASADO EN CONTENIDO (CORRELACIÓN)
#####################################################################

def recomendacion_pelicula(movie=list(df_peliculas['title'])):
    ind_movie = df_peliculas[df_peliculas['title'] == movie].index[0]
    similar_movies = df_dum2.corrwith(df_dum2.iloc[ind_movie, :], axis=1)
    similar_movies = similar_movies.sort_values(ascending=False)
    top_similar_movies = similar_movies.to_frame(name="correlación").iloc[1:11, :]  # omite la misma película
    top_similar_movies['title'] = df_peliculas['title']
    return top_similar_movies

print(interact(recomendacion_pelicula))

interactive(children=(Dropdown(description='movie', options=('Toy Story', 'Jumanji', 'Grumpier Old Men', 'Fath…

<function recomendacion_pelicula at 0x7a8aa3c35e40>


In [29]:
#####################################################################
### 4. RECOMENDACIÓN KNN BASADO EN CONTENIDO (UNA PELÍCULA VISTA)
#####################################################################

# Entrenar modelo KNN
model = neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model.fit(df_dum2)
dist, idlist = model.kneighbors(df_dum2)

# Función interactiva de recomendación
def MovieRecommender(movie_name=list(df_peliculas['title'].value_counts().index)):
    movie_id = df_peliculas[df_peliculas['title'] == movie_name].index[0]
    movie_list_name = []
    for newid in idlist[movie_id]:
        movie_list_name.append(df_peliculas.loc[newid].title)
    return movie_list_name[1:]  # omitir la misma película

print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Planet of the Apes', 'Jungle Book, The', 'S…

<function MovieRecommender at 0x7a8aa29f3060>


In [ ]:
#####################################################################
### 5. RECOMENDACIÓN BASADA EN USUARIO (FILTRADO COLABORATIVO)
#####################################################################

In [37]:
def crear_perfil_usuario(user_id):
    # Filtrar ratings del usuario
    user_ratings = ratings[ratings['userId'] == user_id]

    # Traer las películas que ha calificado
    user_movies = df_dum2.loc[user_ratings['movieId']]

    # Ponderar por la calificación
    user_profile = user_movies.mul(user_ratings['rating'].values, axis=0)

    # Crear un perfil promedio ponderado
    user_profile = user_profile.mean(axis=0)

    return user_profile

In [38]:
def recomendar_por_usuario(user_id, n_recomendaciones=10):
    perfil = crear_perfil_usuario(user_id)

    # Calcular similitud (correlación o distancia euclidiana inversa)
    similitudes = df_dum2.dot(perfil)

    # Eliminar películas ya vistas por el usuario
    peliculas_vistas = ratings[ratings['userId'] == user_id]['movieId'].values
    similitudes = similitudes[~df_dum2.index.isin(peliculas_vistas)]

    # Seleccionar top N
    top_peliculas = similitudes.sort_values(ascending=False).head(n_recomendaciones)

    # Mostrar títulos
    return df_peliculas.loc[top_peliculas.index, ['title']].assign(similitud=top_peliculas.values)

In [42]:
# Crear matriz usuario × película
ratings_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# Calcular similitud entre usuarios
user_similarity = cosine_similarity(ratings_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

# Función de recomendación
def recomendar_por_usuario(user_id):
    # Películas que ya vio el usuario
    peliculas_vistas = ratings_matrix.loc[user_id]
    no_vistas = peliculas_vistas[peliculas_vistas == 0].index

    # Similaridad con otros usuarios
    similares = user_similarity_df[user_id].drop(user_id)  # excluirse a sí mismo

    # Calcular puntaje ponderado por similitud solo para películas no vistas
    ponderado = ratings_matrix.loc[similares.index, no_vistas].T.dot(similares)
    recomendaciones = ponderado.sort_values(ascending=False).head(10)

    # Mostrar títulos
    return df_peliculas[df_peliculas['movieId'].isin(recomendaciones.index)][['movieId', 'title']].drop_duplicates()

# Interfaz interactiva
@interact
def recomendaciones_interactivas(user_id=sorted(ratings['userId'].unique())):
    display(recomendar_por_usuario(user_id))

interactive(children=(Dropdown(description='user_id', options=(np.int64(1), np.int64(2), np.int64(3), np.int64…